In [519]:
import pandas as pd
import yfinance as yf
from tools import atr, sma

In [520]:
def casey_c(close: pd.Series, lookback=5, smoothing=3) -> pd.Series:
    casey = close.pct_change()
    return (
        (
            (casey - casey.rolling(lookback).min())
            / (casey.rolling(lookback).max() - casey.rolling(lookback).min())
            * 100
        )
        .rolling(smoothing)
        .mean()
    )

In [521]:
SYMBOL = "^GSPC"
# SYMBOL = "SPY"

# SYMBOL = "^NDX"
# SYMBOL = "QQQ"


stock = yf.download(SYMBOL)

# some data cleaning
stock = stock[~(stock.High == stock.Low) & ~(stock.Open == stock.Close)]
stock = stock.dropna()

[*********************100%%**********************]  1 of 1 completed


In [522]:
stock["casey"] = casey_c(stock.Close, 5, 3)

In [523]:
# Compute the IBS indicator: (Close - Low) / (High - Low)
stock["ibs"] = (stock.Close - stock.Low) / (stock.High - stock.Low)
stock["atr_low"] = stock.Close - 1.5 * atr(stock, 10)
stock["atr_high"] = stock.Close + 4.5 * atr(stock, 10)


stock["sma"] = sma(stock.Close, 5)
stock["lower_band"] = stock.Low.rolling(10).min()

In [524]:
from backtesting import Strategy


class mean_reversion(Strategy):
    """
    strategy for trend_trading
    """

    lookback = 9
    smoothing = 3
    buy_entry = 35
    buy_exit = 75
    buy_stop = 10
    sma_interval = 5
    ibs_level = 30

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )

    def init(self):
        super().init()

        self.ibs = self.I(lambda: self.data.ibs, name="ibs")
        self.atr_low = self.I(lambda: self.data.atr_low, name="atr_low")
        # self.atr_high = self.I(lambda: self.data.atr_high, name="atr_high")
        self.casey_c = self.I(casey_c, self.data.Close.s, self.lookback, self.smoothing)

    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for existing trade
        for trade in self.trades:
            if trade.is_long:
                # trade.sl = max(trade.sl, self.lower_band[-1])

                # close on next day open
                if self.casey_c[-1] > self.buy_exit:
                    trade.sl = max(trade.sl, _low)
                    # trade.close()

                if self.casey_c[-1] < self.buy_stop:
                    trade.sl = max(trade.sl, _low)
                    # trade.close()

        if len(self.trades) == 0:
            if self.casey_c[-1] < self.buy_entry and self.ibs[-1] < self.ibs_level:
                self.buy(sl=self.atr_low[-1])

In [525]:
from backtesting import Backtest

bt = Backtest(
    stock["2000-01-01":"2011-01-01"],
    mean_reversion,
    cash=100_000,
    commission=0.002,
)
stats = bt.run()
bt.plot(superimpose=False)
stats

Start                     2000-01-03 00:00:00
End                       2010-12-31 00:00:00
Duration                   4015 days 00:00:00
Exposure Time [%]                   74.358047
Equity Final [$]                111646.179219
Equity Peak [$]                 112549.565025
Return [%]                          11.646179
Buy & Hold Return [%]              -13.577326
Return (Ann.) [%]                    1.009089
Volatility (Ann.) [%]               18.467241
Sharpe Ratio                         0.054642
Sortino Ratio                        0.081834
Calmar Ratio                         0.022657
Max. Drawdown [%]                  -44.537953
Avg. Drawdown [%]                   -5.546465
Max. Drawdown Duration     3521 days 00:00:00
Avg. Drawdown Duration      284 days 00:00:00
# Trades                                  187
Win Rate [%]                        39.572193
Best Trade [%]                      22.069061
Worst Trade [%]                    -10.014177
Avg. Trade [%]                    

In [526]:
stats, heatmap = bt.optimize(
    ibs_level=range(5, 50, 5),
    lookback=range(5, 10, 1),
    smoothing=range(1, 5, 1),
    buy_entry=range(10, 40, 5),
    buy_stop=range(10, 40, 5),
    buy_exit=range(70, 85, 5),
    maximize="Equity Final [$]",  ##"Equity Final [$]",  # "Profit Factor"
    max_tries=700,
    random_state=0,
    return_heatmap=True,
)

/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 688 configurations.
  output = _optimize_grid()
/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [527]:
# heatmap
heatmap.sort_values().iloc[-10:]

ibs_level  lookback  smoothing  buy_entry  buy_stop  buy_exit
40         5         4          15         10        75          170142.998048
10         5         4          15         15        70          177006.096562
45         5         4          20         25        70          178706.209737
35         8         3          20         10        80          188177.031633
40         5         4          20         20        70          196874.245988
45         5         4          20         20        70          196874.245988
30         5         4          20         10        80          199176.806035
5          5         4          20         15        80          204349.004358
45         5         4          20         10        75          217871.453405
25         5         4          20         15        75          220132.341545
Name: Equity Final [$], dtype: float64

In [528]:
# pd.DataFrame(heatmap.sort_values()).dropna().iloc[-20:]

In [529]:
# Recommendation
stats["_strategy"]

<Strategy mean_reversion(ibs_level=25,lookback=5,smoothing=4,buy_entry=20,buy_stop=15,buy_exit=75)>

In [530]:
stats = bt.run(**stats._strategy._params)
bt.plot(superimpose=False, open_browser=True)
stats

Start                     2000-01-03 00:00:00
End                       2010-12-31 00:00:00
Duration                   4015 days 00:00:00
Exposure Time [%]                   38.661844
Equity Final [$]                220132.341545
Equity Peak [$]                  235351.71295
Return [%]                         120.132342
Buy & Hold Return [%]              -13.577326
Return (Ann.) [%]                    7.456316
Volatility (Ann.) [%]               12.006417
Sharpe Ratio                         0.621028
Sortino Ratio                         1.05958
Calmar Ratio                         0.609313
Max. Drawdown [%]                  -12.237249
Avg. Drawdown [%]                   -2.339442
Max. Drawdown Duration     1368 days 00:00:00
Avg. Drawdown Duration       57 days 00:00:00
# Trades                                   84
Win Rate [%]                        47.619048
Best Trade [%]                      17.121389
Worst Trade [%]                     -4.362749
Avg. Trade [%]                    

In [531]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,71,15,18,1404.333089,1366.445029,-2690.052264,-0.026979,2000-01-25,2000-01-28,3 days
1,64,61,64,1511.537060,1474.630005,-2362.051500,-0.024417,2000-03-30,2000-04-04,5 days
2,65,72,72,1443.391030,1385.475010,-3764.541301,-0.040125,2000-04-14,2000-04-14,0 days
3,67,73,77,1359.273179,1422.079956,4208.054082,0.046206,2000-04-17,2000-04-24,7 days
4,67,85,94,1417.930176,1450.760010,2199.598893,0.023153,2000-05-04,2000-05-17,13 days
...,...,...,...,...,...,...,...,...,...,...
79,202,2648,2648,1096.037676,1069.434980,-5373.744404,-0.024272,2010-07-16,2010-07-16,0 days
80,202,2649,2669,1068.983676,1072.140015,637.580500,0.002953,2010-07-19,2010-08-16,28 days
81,192,2693,2718,1128.823086,1184.739990,10736.045578,0.049536,2010-09-20,2010-10-26,36 days
82,187,2730,2731,1215.466119,1198.714961,-3132.466584,-0.013782,2010-11-11,2010-11-12,1 days
